# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [25]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
file = pd.read_csv('preprocessed_descriptions.csv', index_col=0)
file['preprocessed_descriptions'] = file['preprocessed_descriptions'].apply(str)
f = (' ').join(file['preprocessed_descriptions'].values)
words = nltk.tokenize.word_tokenize(f)
dict1 = nltk.FreqDist(words)
words = [word for word, freq in dict1.items() if word.isdigit() == False and freq ==1]
words

['meskan',
 'jolla',
 'theatre',
 'transfering',
 'saucethese',
 'casserolei',
 'soupsubmitted',
 '28th2008',
 'lem',
 'burek',
 'europeans',
 'mooing',
 'puffsa',
 'attests',
 'acompaniment',
 'thinkingho',
 'zaardata',
 'baseplease',
 '9th2008',
 'skordalia',
 'skordy',
 'penulum',
 'premixing',
 'dentist',
 'good071908',
 'pitcherfull',
 'mestill',
 'homemadeand',
 'httpshoottocookcomrecipesbakingartisanbreadinfiveminutes',
 'greatusing',
 'yummmmmy',
 'passeddownthegenerations',
 'baconcheeseburger',
 'everthing',
 'cutlerya',
 'applianceadd',
 'terichicken',
 'teribeef',
 'cookereasy',
 'cheffy',
 'frames',
 'protested',
 'recipethere',
 'dhooghe',
 'nickname',
 'cravingtheres',
 'melanies',
 'soiled',
 'fmaily',
 'chesse',
 'insight',
 'awesomefor',
 'wwwglutenfreegirlsblogspotcom',
 'irishness',
 'hershould',
 'fairrecipe',
 'nostrom',
 'humboldt',
 'mulitgrain',
 'fourlayer',
 'collaborated',
 'dedicate',
 'flower7',
 'mimosa',
 'kelbels',
 'emailper',
 'butteryeggy',
 'httpwww

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [26]:
from random import sample
from nltk.metrics import edit_distance
f = (' ').join(file['preprocessed_descriptions'].values)
words = nltk.tokenize.word_tokenize(f)
s = [sample(words, 2) for _ in range(5)]
for i in range(5):
    print(f'Пара №{i+1} {s[i]} ->-> {edit_distance(s[i][0], s[i][1])}')

Пара №1 ['been', 'nice'] ->-> 4
Пара №2 ['i', 'up'] ->-> 2
Пара №3 ['pie', 'this'] ->-> 3
Пара №4 ['served', 'a'] ->-> 6
Пара №5 ['recipe', 'its'] ->-> 5


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [27]:
def f(word,k):
    f = (' ').join(file['preprocessed_descriptions'].values)
    words = nltk.tokenize.word_tokenize(f)
    mini = 10**6
    close_to_word = {i:(edit_distance(word,i)) for i in words}
    return sorted(close_to_word.items(), key=lambda item: item[1])[:k]
f('monday',4)

[('monday', 0), ('sunday', 2), ('today', 2), ('money', 2)]

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [28]:
from nltk.stem import SnowballStemmer
snb_stemmer_ru = SnowballStemmer('russian')

stemmed  = [snb_stemmer_ru.stem(j) for j in words]
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
normalized = [wnl.lemmatize(i) for i in words]
df = pd.DataFrame({'word':words,'stemmed_word':stemmed,'normalized_word':normalized})
df = df.set_index('word')
df[df['stemmed_word']!=df['normalized_word']]

,stemmed_word,normalized_word
word,,
georges,georges,george
its,its,it
children,children,child
friends,friends,friend
popsicles,popsicles,popsicle
...,...,...
sites,sites,site
its,its,it
roots,roots,root


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [29]:
from nltk.corpus import stopwords

descriptions = ' '.join(list(file['preprocessed_descriptions']))
total_words = list(nltk.word_tokenize(descriptions.lower()))
words = set(nltk.word_tokenize(descriptions.lower()))

stop_words = set(stopwords.words('english'))
file['preprocessed_descriptions'] = file['preprocessed_descriptions'].\
apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
words3 = [word for word in total_words if word not in stop_words]
print('Доля от общего кол-ва, которую составляли стоп слова:', 1 - len(words3)/len(total_words))

print('до удаления стоп-слов')
for k, v in nltk.FreqDist(total_words).most_common(10):
    print(f'{k} - {v}')
print('---------------------')
print('после удаления стоп-слов')
for k, v in nltk.FreqDist(words3).most_common(10):
    print(f'{k} - {v}')
      

Доля от общего кол-ва, которую составляли стоп слова: 0.45646494716721886
до удаления стоп-слов
the - 40072
a - 34951
and - 30245
this - 26859
i - 24836
to - 23471
is - 20285
it - 19756
of - 18364
for - 15939
---------------------
после удаления стоп-слов
recipe - 14871
make - 6326
time - 5137
use - 4620
great - 4430
like - 4167
easy - 4152
one - 3872
made - 3810
good - 3791


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [30]:
from sklearn.feature_extraction.text import (CountVectorizer, TfidfVectorizer)
a = (file.sample(5))
arr_name =  a.name
arr_desc = a.preprocessed_descriptions
# создание векторизатора:
tv = TfidfVectorizer()
# векторизуем корпус:
corpus_tv = tv.fit_transform(arr_desc)
corpus_tv = corpus_tv.toarray()
corpus_tv

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.5       , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.5       , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.5       , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.5       , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.14547154, 0.14547154, 0.14547154, 0.1

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [41]:
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import sent_tokenize
txt = sent_tokenize('''Считайте слова из файла litw-win.txt и запишите их в список words. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка words. Считайте, что в слове есть опечатка, если данное слово не содержится в списке words.''')
res = dfs['preprocessed_descriptions'].values.tolist()
vectorizer = TfidfVectorizer()
vectorizer.fit_transform(res)
vctrzr = CountVectorizer()
vctrzd_txt = vctrzr.fit_transform(txt).toarray()
vctrzd = vectorizer.fit_transform(res).toarray()
dfs = file.sample(5)
df_11 = pd.DataFrame(index = dfs.name, columns = dfs.name)
for k,v in enumerate(dfs.name):
    for i,j in enumerate(dfs.name):
        df_11.at[v,j] = 1 - cosine(vctrzd[k], vctrzd[i])
df_11 = df_11.reset_index().rename(columns = {'name':' '})
df_11 = df_11.set_index(' ')
df_11

name,mike s italian meatballs,mediterranean mixed pepper salad,skillet chicken with black beans and rice,banana tea,fluffy banana graham pancakes
,,,,,
mike s italian meatballs,1,0.0,0.060324,0.0,0.052743
mediterranean mixed pepper salad,0.0,1,0.0,0.0,0.0
skillet chicken with black beans and rice,0.060324,0.0,1,0.0,0.027622
banana tea,0.0,0.0,0.0,1,0.034377
fluffy banana graham pancakes,0.052743,0.0,0.027622,0.034377,1


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

In [42]:
val = df_11[df_11 != 1].max().values
df_11[df_11 == val]

name,mike s italian meatballs,mediterranean mixed pepper salad,skillet chicken with black beans and rice,banana tea,fluffy banana graham pancakes
,,,,,
mike s italian meatballs,NaN,0.0,0.060324,NaN,0.052743
mediterranean mixed pepper salad,NaN,NaN,NaN,NaN,NaN
skillet chicken with black beans and rice,0.060324,0.0,NaN,NaN,NaN
banana tea,NaN,0.0,NaN,NaN,NaN
fluffy banana graham pancakes,NaN,0.0,NaN,0.034377,NaN
